In [1]:
# Input setup

loan_app_name = "loan_application_form.docx"
cr_name = "industrial_license.pdf"
il_name = "industrial_license.png"

In [2]:
from scripts.document_ext import Data_extractor

/home/ubuntu/loan_analysis_app/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = Data_extractor().get(loan_app_name, cr_name, il_name)

test-1


convert /home/ubuntu/loan_analysis_app/data/loan_application_form.docx -> /home/ubuntu/loan_analysis_app/data/loan_application_form.pdf using filter : writer_pdf_Export
Overwriting: /home/ubuntu/loan_analysis_app/data/loan_application_form.pdf
D1
D2
D3
D4


In [ ]:
import os
# If you're using the Python REPL tool, note it has moved to `langchain_experimental`
from langchain_experimental.tools import PythonREPLTool
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
import os
from dotenv import load_dotenv
load_dotenv()

# Now retrieve the key from the environment
openai_api_key = os.getenv("OPENAI_API_KEY")
# -------------------------------------------------------------------
# 1. System instructions (the multi-step prompt).
# -------------------------------------------------------------------
system_instructions = """\
You are a smart business analysis tool provided with 'Tables of loan application' and 'Market Data'.
You have access to a math tool for calculations when necessary.

Follow these steps:
1. Extract Table 2.8 and its data from the 'Tables of loan application'.
2. Extract corresponding market data for each product in Table 2.8.
3. Calculate the growth rate for each product in Table 2.8 based on the anual sales projection for each year related to a specific product.
3. compare the anual growth rate per year calculated above with the anual growth rate provided in the  "market data" for each product.
   - If the calculate market growth rate does not match and is very off with the provided numbers in market data for that product, flag it.
4. Request clarifications or revisions:
   - For descripancy in the growth rate, ask for an explanation and reasoning.
   - Also report if the growth rate is missing any product in table 2.8 and is present in market data.
5. Associate each request with Table 2.8 and specific products  and fields by year where discrepancies were found.
6. **Output** only valid JSON objects list with below structure:
   [
   {{
     "Perspective": "Market",
     "Request": "<Your text for first request here>"
   }},
    {{
     "Perspective": "Market",
     "Request": "<Your text for second request here>"
   }},
   ....
   ]

Examples of output:
[
{{
  "Perspective": "Market",
  "Request": "The market growth rate for product A is 5% while your projected growth rate is 1% for year 2025 according to the projected sales data in table 2.8. Please provide the basis and reasoning for this discrepancy."
}},
{{
  "Perspective": "Market",
  "Request": "The market growth rate for product B is 3% while your projected growth rate is -2% for year 2025 according to the projected sales data in table 2.8. Please provide the basis and reasoning for this discrepancy."
}},
]

No additional text or formatting is allowed beyond these JSON objects. The data for analysis is given as under;
"""

# -------------------------------------------------------------------
# 2. Human template
# -------------------------------------------------------------------
human_template = """\
'Tables of loan application':
{tables}

'Market Data':
{market_data}
"""

# -------------------------------------------------------------------
# 3. LLM (OpenAI Chat)
# -------------------------------------------------------------------
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.0,
    openai_api_key=openai_api_key
)

# -------------------------------------------------------------------
# 4. Define the Python REPL tool (if needed for calculations).
# -------------------------------------------------------------------
python_repl_tool = PythonREPLTool()

# -------------------------------------------------------------------
# 5. Initialize the agent
# -------------------------------------------------------------------
tools = [python_repl_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)

# -------------------------------------------------------------------
# 6. A function to run the agent using a single string input.
# -------------------------------------------------------------------
def analyze_growth_discrepancies(tables: str, market_data: str) -> str:
    """
    Takes the text for 'Tables of loan application' and 'Market Data',
    and returns the JSON objects as a string (with no extra text)
    according to the instructions.
    """
    # Combine both system instructions and the user data into one string:
    final_prompt = (
        f"{system_instructions}\n"
        + human_template.format(tables=tables, market_data=market_data)
    )

    # Pass the final prompt to the agent via the `run` method
    # which expects either a single string or {'input': ...}
    response = agent.run({"input": final_prompt, "chat_history": []})
    return response

# -------------------------------------------------------------------
# 7. Example usage
# -------------------------------------------------------------------
if __name__ == "__main__":
    example_tables = data['loan_app_tables']
    example_market_data = data["market_data"]

    final_json_output = analyze_growth_discrepancies(example_tables, example_market_data)
    print(final_json_output)


/tmp/ipykernel_133130/2046508258.py:71: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(




> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": [
        {
            "Perspective": "Market",
            "Request": "The market growth rate for Paper Cups is 5% while your projected growth rate is 4% for year 2025 according to the projected sales data in table 2.8. Please provide the basis and reasoning for this discrepancy."
        },
        {
            "Perspective": "Market",
            "Request": "The market growth rate for Paper Bowels is 4% while your projected growth rate is 4% for year 2025 according to the projected sales data in table 2.8. Please provide the basis and reasoning for this discrepancy."
        }
    ]
}
```

> Finished chain.
[{'Perspective': 'Market', 'Request': 'The market growth rate for Paper Cups is 5% while your projected growth rate is 4% for year 2025 according to the projected sales data in table 2.8. Please provide the basis and reasoning for this discrepancy.'}, {'Perspective': 'Market', 'R

In [8]:
prompts = [
    {
      "task_number": "1",
      "task": "missing_data",
      "prompt": "You are a smart data analysis tool provided with “Tables of loan application” in four categories:\n1. PROJECT DATA (Included Tables: GENERAL INFORMATION, PROJECT PROFILE, PROJECT OWNERSHIP, PROJECT REPRESENTATIVE, A BRIEF INFORMATION ON PROJECT OWNER (S), BANK INFORMATION),\n2. MARKETING INFORMATION (Included Tables: 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 2.10, 2.11, 2.12, 2.13, 2.14. 2.15),\n3. TECHNICAL INFORMATION (Tables Included: 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 3.10, 3.11, 3,12, 3.13, 3.14, 3.15, 3.16, 3.17),\n4. FINANCIAL INFORMATION (Tables Included: 4.1, 4.2, 4.3).\n\nYour job is to identify each table mentioned along with its provided data, perform an exhaustive analysis of each table, identify any missing or incomplete fields and cells (including placeholders like, “—”, whitespace “”), detect missing or required attachment documents, categorize each finding by perspective, and output each finding in a numbered list format.\n\n1. **Exhaustive Analysis of All Tables and Fields**\n   - Identify and extract each table and its corresponding data, inspect every row and cell across all tables.\n   - Determine if each field or cell is properly filled with valid data or if it appears to be missing, incomplete, or contains placeholder text (e.g., “N/A”, “—”, “ ”).\n\n2. **Identify Missing or Incomplete Fields and Attachments**\n   - Consider a field “missing” if:\n     - It is blank or contains only whitespace.\n     - It contains a placeholder like “N/A” or “—”.\n     - It references an attachment that has not been provided (e.g., missing PDF or supporting document).\n   - Consider a field “incomplete” if it has partial data that indicates further details are required (e.g., partial references or truncated text).\n\n3. **Categorize Each Finding by Perspective**\n   - Credit: Any field related to financial or credit details, such as interest rates, repayment terms, financial statements, or credit history documents.\n   - Market: Any field related to market studies, marketing strategies, customer segmentation, demand analysis, competition, or commercial feasibility.\n   - Technical: Any field related to official documents, certifications, or technical feasibility requirements (e.g., building permits, design schematics, or compliance certificates).\n\n4. **Output Format**\n   - For each missing or incomplete field (or table, if entire sections are missing), output the following numbered format:\n     Outputs:\n     1. `<Make request to complete a field or fill some table with clearly mentioning the table/field details>` (`<Associated Category>`)\n     2. `<Make request to complete a field or fill some table with clearly mentioning the table/field details>` (`<Associated Category>`)\n     3. `<Make request to complete a field or fill some table with clearly mentioning the table/field details>` (`<Associated Category>`)\n     …\n   - No additional commentary or explanation should be included—only the numbered requests with categories."
    },
    {
      "task_number": "2",
      "task": "Cross-validation with attachment Doc",
      "prompt": "You are an advanced data analysis tool tasked with processing the following data sources:\n\n- **Tables of Loan Application**\n- **Commercial Registration Document**\n- **Industrial License Document**\n\nYour objectives are:\n\n1. **Extract Identification Numbers:**\n   - Retrieve the Commercial Registration Number (رقم السجل) from both the **Tables of Loan Application** (From table: GENERAL INFORMATION) and the **Commercial Registration Document**.\n   - Retrieve the Industrial License Number (رقم الترخيص) from both the **Tables of Loan Application** and the **Industrial License Document**.\n\n2. **Validate Data Consistency:**\n   - Compare the extracted **Commercial Registration Numbers** from the tables with those from the Commercial Registration Document.\n   - Compare the extracted **Industrial License Numbers** from the tables with those from the Industrial License Document.\n\n3. **Handle Discrepancies:**\n   - If any **Commercial Registration Number** or **Industrial License Number** from tables (1. PROJECT DATA) is missing or does not match with the corresponding document, generate a request for the user to provide the correct numbers.\n\n4. **Generate Response:**\n   - For each verification (Commercial Registration Number and Industrial License Number), return the output as a list of responses formatted as follows:\n\n```\nOutputs:\n1. <Some Request> (<Associated Category>)\n2. <Some Request> (<Associated Category>)\n3. <Some Request> (<Associated Category>)\n...\n```\n\n**Example Outputs:**\n\n```\nOutputs:\n1. The industrial license number 22234334343 in \"Project Profile\" does not match with the provided industrial license document.\n2. The commercial registration number 45345345 in \"General Information\" does not match with the provided commercial registration document.\n3. (empty string if the numbers match exactly)\n```\n\n**Important:** The output must contain the information exactly as requested, in the format specified. If the numbers match exactly, return an empty string (i.e., no request). Do not include any additional commentary, explanations, or text outside of the requested output."
    },
    {
      "task_number": "3",
      "task": "Identifying Mismatched Data in Loan Applications",
      "prompt": "You are a smart data analysis tool. You are provided with \"Tables of loan application\" in four categories:\n1. PROJECT DATA\n   - Included Tables: GENERAL INFORMATION, PROJECT PROFILE, PROJECT OWNERSHIP, PROJECT REPRESENTATIVE, \n     A BRIEF INFORMATION ON PROJECT OWNER(S), BANK INFORMATION\n2. MARKETING INFORMATION\n   - Included Tables: 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 2.10, 2.11, 2.12, 2.13, 2.14, 2.15\n3. TECHNICAL INFORMATION\n   - Included Tables: 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 3.10, 3.11, 3.12, 3.13, 3.14, 3.15, 3.16, 3.17\n4. FINANCIAL INFORMATION\n   - Included Tables: 4.1, 4.2, 4.3\n\nFollow the steps below to produce the requested output JSON. If you do not see any data for a particular table, assume that table is empty or missing. Make sure to handle variations in field naming (synonyms, slightly different labeling, etc.).\n\n### Instructions\n\n1. **Identify and Extract Tables**  \n   - Parse all provided data carefully.  \n   - Identify each mentioned table (e.g., \"Table 2.1,\" \"GENERAL INFORMATION,\" etc.) and extract fields.  \n   - If a table is referenced but no data is provided, treat it as empty.\n\n2. **Organize Extracted Data (Temporary/Intermediate) attributed to each table**  \n   - Extract data attributed to each table in an organized way. n\n2. **Analyze and Identify Inconsistencies**  \n   - Compare same fields across all tables for values that should remain consistent but differ (e.g., \"Project Location\" across multiple tables).  \n   - Consider Abbreviations as right (e.g., \"LLC\" vs. \"Limitted Liability Company\").\n\n4. **Inconsistency Detection**  \n   - For each conflict:  \n     - **Which table(s) or fields** are involved in the inconsistency?  \n     - **What are the differing values?**  \n\n5. **Categorize by Perspective**  \n   - **Credit Perspective**: Inconsistencies related to financial data, credit, loan amounts, bank info, or payment schedules.  \n   - **Market Perspective**: Inconsistencies related to marketing strategy, target audience, location, or distribution.  \n   - **Technical Perspective**: Inconsistencies related to required documents, certifications, operational feasibility, or technical requirements.\n\n6. **Output**  \n   - For **each** identified inconsistency, return **only** a JSON object in the following format (one JSON object per inconsistency or grouped logically if multiple tables/fields are involved):\n     ```json\n     {\n       \"Perspective\": \"<Credit | Market | Technical>\",\n       \"Request\": \"<Request clarification with table/field details>\"\n     }\n     ```\n   - No additional text, commentary, or explanations outside of these JSON objects.  \n   - If there are **no** inconsistencies, output an empty list:\n     ```json\n     []\n     ```\n\n### Additional Notes\n- When encountering synonyms or different labels (e.g., “Target Region” vs. “Market Region”), normalize or compare them as the same field.  \n- If a single field is inconsistent in multiple places, you may group them in one JSON object or produce multiple JSON objects, as long as it remains clear.  \n- Do **not** print any text beyond the required JSON objects or an empty list (`[]`)."
    },
    {
      "task_number": "4",
      "task": "Identifying incomplete information",
      "prompt": "You are an advanced data analysis tool that reviews loan application tables for missing or unclear business details. You are provided with \"Tables of loan application\" in four categories:\n1. PROJECT DATA\n   - Included Tables: GENERAL INFORMATION, PROJECT PROFILE, PROJECT OWNERSHIP, PROJECT REPRESENTATIVE, \n     A BRIEF INFORMATION ON PROJECT OWNER(S), BANK INFORMATION\n2. MARKETING INFORMATION\n   - Included Tables: 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 2.10, 2.11, 2.12, 2.13, 2.14, 2.15\n3. TECHNICAL INFORMATION\n   - Included Tables: 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 3.10, 3.11, 3.12, 3.13, 3.14, 3.15, 3.16, 3.17 \n\n Your Tasks\n\n1. Parse and Identify the Target Tables for this analysis\n   - Locate 'PROJECT PROFILE', Table 2.9, Table 3.1, and Table 3.5\" within the provided data.\n   - Identify all fields, sentences, or values in each table/section.\n\n2. Identify Incomplete or Unclear Information\n   - Look for incomplete sentences or statements that do not clearly convey the intended business meaning.\n   - Specifically check whether keywords such as \"trading experience,\" \"industrial experience,\" \"market growth,\" and \"imports\" are missing from any project descriptions in:\n     - Project Profile (page 1)\n     - Table 2.9\n   - If these keywords are absent or only partially addressed, consider the entries incomplete or unclear.\n\n3. Categorize by Perspective\n   - Use \"Project Perspective\" for missing or unclear details about the project setup, strategic factors, or required business background (e.g., \"trading experience,\" \"industrial experience,\" \"market growth,\" \"imports,\" etc.).\n   - Use \"Market Perspective\" for missing or unclear market-related details (e.g., demand/supply analysis, growth projections, competitor assessments).\n   - Use \"Cost Perspective\" for missing cost details (e.g., only a total building cost in Table 3.5 without a breakdown).\n\n4. Request Clarification and Details\n   - For every field you find to be incomplete or unclear, output one JSON object containing:\n     {\n       \"Perspective\": \"<Project|Market|Cost>\",\n       \"Request\": \"A concise request for more information, explicitly mentioning the table name (or 'Project Profile, page 1') and the specific field.\"\n     }\n   - Mention the missing keywords if they are relevant. For instance, if the user wrote \"I want to open a factory to sell product x\" in the Project Profile (page 1) but did not mention trading or industrial experience, the model must explicitly request those details.\n   - If you see only a total building cost in Table 3.5 without a breakdown, request a detailed cost breakdown (e.g., materials, labor, equipment).\n\n5. Output Formatting\n   - Do not include any text outside of valid JSON objects.\n   - You may output multiple JSON objects (one for each incomplete or unclear entry) either sequentially or as an array—depending on user preference.\n\n6. Edge Cases\n   - No Incomplete Fields: If nothing is missing or unclear, produce no JSON objects or an empty array ([]) based on user preference.\n   - Contradictory Data: If conflicting information appears in different tables or sections, treat it as unclear and request clarification.\n   - Multiple Gaps in a Single Field: If multiple details (e.g., trading experience and market growth) are missing from one statement, you may either combine them into a single JSON \"Request\" or list separate ones; remain consistent.\n\n### Illustrative Examples\n\n1. Missing Business Keywords\n   - Found in Project Profile (page 1): \"I want to open a factory to sell Product X.\"\n   - Missing explicit mention of trading/industrial experience, market growth, imports, etc.\n   - JSON Object:\n     {\n       \"Perspective\": \"Project\",\n       \"Request\": \"Could you please provide more details in the Project Profile (page 1) about your trading experience, industrial experience, market growth, and imports for Product X?\"\n     }\n\n2. Incomplete Market Analysis\n   - Found in Table 2.9: \"The local market for Product Y is big.\"\n   - Missing data on competitor analysis, growth projections, etc.\n   - JSON Object:\n     {\n       \"Perspective\": \"Market\",\n       \"Request\": \"Kindly clarify the market growth projections, competitor assessments, and any import factors in Table 2.9 for Product Y.\"\n     }\n\n3. Total Building Cost Without Breakdown\n   - Found in Table 3.5: \"Total building cost: 200,000.\"\n   - No itemized list of materials, labor, or equipment.\n   - JSON Object:\n     {\n       \"Perspective\": \"Cost\",\n       \"Request\": \"Please provide a breakdown of the building costs in Table 3.5, including materials, labor, and equipment details.\"\n     }\n\nUse this prompt exactly as specified to ensure your output consists solely of JSON objects that capture each missing or unclear detail, referencing the relevant table or section name and requesting specific clarifications."
    },
    {
      "task_number": "5",
      "task": "Data Generation Capability",
      "prompt": "You are an advanced data analyst. Analyze the provided \"Tables of Loan Application\" and perform the following tasks:\n\n1. **Identify and Extract Table 2.4:**\n   - Locate \"Table 2.4 Target Market(s)\" within the provided tables.\n   - Extract the contents of Table 2.4 accurately.\n\n2. **Identify Selected Target Markets:**\n   - From Table 2.4, list all selected target markets (e.g., wholesalers, factories, distributors, etc.).\n\n3. **Examine Each Selected Target Market:**\n   - For each identified target market, assess the available data.\n   - Determine if additional data tables are required to support analysis for each target market.\n\n4. **Determine Additional Data Tables Needed:**\n   - For each target market that lacks sufficient data, specify which additional tables are necessary.\n   - For example, if \"Wholesales\" is a target market and historical or projected sales data is missing, indicate the need for those specific tables.\n\n5. **Generate JSON Output:**\n   - For each target market requiring additional data, create a JSON object with the following structure:\n\n     ```json\n     {\n       \"Perspective\": \"Market\",\n       \"Request\": \"A clear and consise request based on the above analysis for the additional tables/data for the Target Market Selected\"\n     }\n     ```\n\n   - Ensure that only JSON objects are returned without any additional commentary or text.\n\n**Example:**\n\nIf the client’s target market is \"Wholesales\" and historical or projected sales data is not provided in Table 2.4, the output should be:\n\n```json\n{\n  \"Perspective\": \"Market\",\n  \"Request\": \"Please provide the historical sales data (by year) and the projected sales data (by year) for Wholesales, as referenced in Table 2.4.\"\n}\n```"
    },
    {
      "task_number": "6",
      "task": "Business judgment intelligence",
      "prompt": "You are a smart business analysis tool provided with 'Tables of loan application' and 'Market Data'.\nYou have access to a math tool for calculations when necessary.\n\nFollow these steps:\n1. Extract Table 2.8 and its data from the 'Tables of loan application'.\n2. Extract corresponding market data for each product in Table 2.8.\n3. Convert the data for each year to percentage e.g: \n3. Identify and analyze discrepancies by calculating the growth rate from table 2.8. compare it to the growth rate per year provided in market data for each product.\n   - If the calculate market growth rate does not match and is very off with the provided numbers in market data for that product, flag it.\n4. Request clarifications or revisions:\n   - For descripancy in the growth rate, ask for an explanation and reasoning.\n   - Also report if the growth rate is missing any product in table 2.8 and is present in market data.\n5. Associate each request with Table 2.8 and specific products  and fields by year where discrepancies were found.\n6. **Output** only valid JSON objects list with below structure:\n   [\n   {{\n     \"Perspective\": \"Market\",\n     \"Request\": \"<Your text for first request here>\"\n   }},\n    {{\n     \"Perspective\": \"Market\",\n     \"Request\": \"<Your text for second request here>\"\n   }},\n   ....\n   ]\n\nExamples of output:\n[\n{{\n  \"Perspective\": \"Market\",\n  \"Request\": \"The market growth rate for product A is 5% while your projected growth rate is 1% for year 2025 according to the projected sales data in table 2.8. Please provide the basis and reasoning for this discrepancy.\"\n}},\n{{\n  \"Perspective\": \"Market\",\n  \"Request\": \"The market growth rate for product B is 3% while your projected growth rate is -2% for year 2025 according to the projected sales data in table 2.8. Please provide the basis and reasoning for this discrepancy.\"\n}},\n]\n\nNo additional text or formatting is allowed beyond these JSON objects. The data for analysis is given as under;"
    }
  ]
  

In [21]:
import os
import json
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import initialize_agent, AgentType

class ai_tool:
    def __init__(self, model="gpt-4o", temperature=0):
        """
        Initialize the SmartAnalysisAgent with model configuration and validate OpenAI API key.
        """
        self._load_and_validate_api_key()
        self.llm = ChatOpenAI(model=model, temperature=temperature)
        self.math_agent = initialize_agent(
            tools=[PythonREPLTool()],
            llm=self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            # handle_parse_errors=True,
            verbose=True
        )

    @staticmethod
    def _load_and_validate_api_key():
        """
        Load and validate the OpenAI API key from the .env file.
        """
        load_dotenv()
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("OpenAI API key not found. Please set it in your .env file.")
        os.environ["OPENAI_API_KEY"] = api_key

    def analyse(self, inputs: dict):
        """
        Run an LLM-based analysis on tables provided in markdown format.
        
        Args:
            inputs (dict): A dictionary containing:
                - "tables": List of tables in markdown format as strings.
                - "analysis_guidelines": Guidelines for the analysis.
                - "output_guidelines": Guidelines for the output formatting.

        Returns:
            str: Analysis result from the LLM.
        """
        # Validate input keys
        step_1_keys = {"tables", "prompt", "industrial_licence", "commercial_registration", "market_data"}
        step_2_keys = {"compile_prompt", "input_payload"}

        if step_1_keys.issubset(inputs.keys()):
            if inputs["prompt"]["task_number"]=="2":
                # Combine inputs into a structured prompt
                prompt_template = """
                {prompt}

                **Tables of loan application**:
                {tables}

                **Commercial Registration**:
                {commercial_registration}

                **Industrial License**:
                {industrial_licence}
                """
                prompt_input = ["tables", "prompt", "industrial_licence", "commercial_registration"]
                if isinstance(inputs["tables"], dict):
                    tables = json.dumps(inputs["tables"]['1. PROJECT DATA'])
                else:
                    tables = inputs["tables"]

                llm_input = {
                "prompt": inputs["prompt"]["prompt"],
                "tables": tables,
                "commercial_registration": "\n\n"+ inputs["commercial_registration"],
                "industrial_licence": "\n\n"+ inputs["industrial_licence"]
                }

            elif inputs["prompt"]["task_number"]=="4":
                    # Combine inputs into a structured prompt
                    prompt_template = """
                    {prompt}

                    **Tables of loan application**:
                    {tables}
                    """
                    prompt_input = ["tables", "prompt"]
                    if isinstance(inputs["tables"], dict):
                        tables = json.dumps({x:y for x,y in inputs["tables"].items() if x in ['1. PROJECT DATA', '2. MARKETING INFORMATION', "3. TECHNICAL INFORMATION"]})
                    else:
                         tables = inputs["tables"]
                    llm_input = {
                    "prompt": inputs["prompt"]["prompt"],
                    "tables": tables
                    }

            elif inputs["prompt"]["task_number"]=="5":
                    # Combine inputs into a structured prompt
                    prompt_template = """
                    {prompt}

                    **Tables of loan application**:
                    {tables}
                    """
                    prompt_input = ["tables", "prompt"]
                    if isinstance(inputs["tables"], dict):
                        tables = json.dumps({x:y for x,y in inputs["tables"].items() if x in ['2. MARKETING INFORMATION']})
                    else:
                         tables = inputs["tables"]
                    llm_input = {
                    "prompt": inputs["prompt"]["prompt"],
                    "tables": tables,
                    }

            elif inputs["prompt"]["task_number"]=="6":
                # Combine inputs into a structured prompt
                prompt_template = """
                {prompt}

                **Tables**:
                {tables}

                **Market Data**:
                {market_data}
                """
                prompt_input = ["tables", "prompt", "market_data"]
                if isinstance(inputs["tables"], dict):
                    tables = json.dumps({x:y for x,y in inputs["tables"].items() if x in ['1. PROJECT DATA', '2. MARKETING INFORMATION', "3. TECHNICAL INFORMATION"]})
                else:
                     tables = inputs["tables"]
                llm_input = {
                "prompt": inputs["prompt"]["prompt"],
                "tables": tables,
                "market_data": "\n\n"+ inputs["market_data"]
                }
                
            else:
                # Combine inputs into a structured prompt
                prompt_template = """
                {prompt}

                **Tables**:
                {tables}
                """
                prompt_input = ["tables", "prompt"]
                if isinstance(inputs["tables"], dict):
                     tables = json.dumps(inputs["tables"])
                else:
                     tables = inputs["tables"]
                
                llm_input = {
                "prompt": inputs["prompt"]["prompt"],
                "tables": tables,
                }

        elif step_2_keys.issubset(inputs.keys()):
                # Combine inputs into a structured prompt
                prompt_template = """
                {prompt}

                **Input Payload**:
                {input_payload}
                """
                prompt_input = ["input_payload", "prompt"]
                llm_input = {
                "prompt": inputs["compile_prompt"],
                "input_payload": "\n\n".join(inputs["input_payload"]),
                }

        elif not step_1_keys.issubset(inputs.keys()):
            raise ValueError(f"Input dictionary must contain keys: {step_1_keys}")
        else:
            raise ValueError(f"Input dictionary must contain keys: {step_1_keys}")
        
        prompt = PromptTemplate(
            input_variables=prompt_input,
            template=prompt_template.strip()
        )
        if inputs["prompt"]["task_number"]=="6":
             prompt_formatted = prompt.format(prompt=llm_input["prompt"],
                                               tables=llm_input["tables"],
                                                 market_data=llm_input["market_data"])
             response = self.math_agent.run({"input": prompt_formatted, "chat_history": []})
             return response
        else:
            chain = prompt | self.llm
            result = chain.invoke(input=llm_input)
            return result.content

In [22]:
ai_agent = ai_tool()
l_appt = data["loan_app_tables"]
il = data["industrial_licence"]
cr = data["commercial_registration"]
md = data["market_data"]

print("Data Extraction Completed!")

input = prompts
all_results = []
for index, prompt in enumerate([input[5]]):
    inputs = {
    "prompt": prompt,
    "tables": l_appt,
    "industrial_licence": il,
    "commercial_registration": cr,
    "market_data": json.dumps(md, indent=4)
    }

    analysis_result = ai_agent.analyse(inputs)
    all_results.append(analysis_result)


Data Extraction Completed!


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": [
        {
            "Perspective": "Market",
            "Request": "Please provide the basis and reasoning for your projected 0% growth rate for Paper Bowels in 2026 as mentioned in Table 2.8, while the market shows a 4% growth rate."
        },
        {
            "Perspective": "Market",
            "Request": "Please revise your sales estimates for Paper Bowels in 2026 to include an appropriate growth percentage in Table 2.8, as the market shows a 4% growth rate."
        }
    ]
}
```

> Finished chain.


In [23]:
all_results[0]

[{'Perspective': 'Market',
  'Request': 'Please provide the basis and reasoning for your projected 0% growth rate for Paper Bowels in 2026 as mentioned in Table 2.8, while the market shows a 4% growth rate.'},
 {'Perspective': 'Market',
  'Request': 'Please revise your sales estimates for Paper Bowels in 2026 to include an appropriate growth percentage in Table 2.8, as the market shows a 4% growth rate.'}]